In [1]:
# In Jupyter Notebooks, it allows you to work with Spark without explicitly configuring the environment, 
# which makes it easier to use Spark for data processing and analysis within the Jupyter environment.
# or when you want to use Spark in an environment where it is not directly available, such as in a Jupyter Notebook
import findspark
findspark.init()

## import required modules

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import *
import pyspark.sql.functions as F
import warnings
from pyspark.sql.types import *

warnings.filterwarnings('ignore')

In [3]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
!java -version

java version "1.8.0_381"
Java(TM) SE Runtime Environment (build 1.8.0_381-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.381-b09, mixed mode)


## Initializing Spark Session

In [5]:
# Initializing a Spark session
spark = SparkSession.builder \
    .appName('BBG Case Study Case 1') \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

In [6]:
# You can check the number of partitions using the following command
spark_partitions = spark.sparkContext.defaultParallelism
print("Number of partitions:", spark_partitions)

Number of partitions: 12


# Case 1.1

### Loading data from CSV files

In [7]:
# Loading the sales data into Spark Cluster
sales_df = spark.read.csv('F:/usecase/DATA/sales_and_traffic_data.csv', header=True)
sales_df.show(5, truncate=False)

+---+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------------------+-----------+
|#  |child_asin|sessions|page_views|units_ordered|units_ordered_b2b|ordered_products_sale|ordered_products_sales_b2b|total_ordered_items|total_ordered_items_b2b|region|shop_name            |report_date|
+---+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------------------+-----------+
|1  |B00H5155LG|1       |1         |0            |0                |116.64               |62.49                     |0                  |0                      |UK    |Elektronik-Star      |2022-03-20 |
|2  |B002P7L4R4|1       |2         |0            |0                |121.55               |227.24                    |0                  |0                      |UK    |Elektronik-Star     

In [8]:
sales_df.count()

5000

In [9]:
# Loading the sales data into Spark Cluster
shop_mapping_df = spark.read.csv('F:/usecase/DATA/amazon_shop_mapping.csv', header=True)
shop_mapping_df.show(5, truncate=False)

+---------+-------+----+--------+
|shop_name|country|id  |currency|
+---------+-------+----+--------+
|Flyweight|IT     |5823|EUR     |
|Flyweight|SE     |4213|SEK     |
|Flyweight|PL     |621 |PLN     |
|Flyweight|TR     |8772|TRY     |
|Flyweight|FR     |9257|EUR     |
+---------+-------+----+--------+
only showing top 5 rows



In [10]:
shop_mapping_df.count()

1082

### Setting up the conversion rate as no free conversion api found so far

In [11]:
conversion_rate = 55

### Join dataframes and apply currency conversion

In [12]:
# Join the two dataframes on shop_name column
joined_df = sales_df.alias("df1").join(broadcast(shop_mapping_df).alias("df2"), on="shop_name", how="inner")

### Select required columns and drop redundant columns

In [13]:
joined_df = joined_df.select("df1.child_asin", "df1.sessions", "df1.page_views", "df1.units_ordered",
                                     "df1.units_ordered_b2b", "df1.ordered_products_sale", "df1.ordered_products_sales_b2b",
                                     "df1.total_ordered_items", "df1.total_ordered_items_b2b", "df1.region",
                                     "df1.shop_name", "df1.report_date", "df2.country",
                                     "df2.id", "df2.currency")
# Show the converted data
joined_df.show(2)


+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+
|child_asin|sessions|page_views|units_ordered|units_ordered_b2b|ordered_products_sale|ordered_products_sales_b2b|total_ordered_items|total_ordered_items_b2b|region|shop_name|report_date|country|  id|currency|
+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+
|B07KNDZF8Z|       1|         1|            0|                0|               242.25|                     73.52|                  0|                      0|    IT|  Feelino| 2022-02-02|     UK|6767|     GBP|
|B07KNDZF8Z|       1|         1|            0|                0|               242.25|                     73.52|                  0|                      0|    IT|

In [14]:
joined_df.count()

1458

In [15]:
import requests
import json

@udf(returnType=FloatType())
def convert_currency(amount, from_currency, to_currency):
    # Constructing the API URL with the parameters
    API_KEY = '21aed5d93748d017cb953a8b4d46248a'
    url = f"http://api.currencylayer.com/convert?access_key={API_KEY}&from={from_currency}&to={to_currency}&amount={amount}"
    # Sending a GET request and get the response
    #response = requests.get(url)
    # Parsing the JSON data
    #data = json.loads(response.text)
    # Get the converted amount
    data = {}
    data["result"] = 55.0
    if 'result' in data:
        # Get the converted amount
        return data["result"]
    #return data["result"]


In [16]:
from pyspark.sql.functions import col, lit
from pyspark.sql import functions as F

# Apply the user-defined function to convert sales columns to EUR
joined_df = joined_df.withColumn("ordered_products_sale_eur", convert_currency(F.col("ordered_products_sale"), F.col("currency"), F.lit("EUR")))

# Show the result
joined_df.show(5, truncate=False)


+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+-------------------------+
|child_asin|sessions|page_views|units_ordered|units_ordered_b2b|ordered_products_sale|ordered_products_sales_b2b|total_ordered_items|total_ordered_items_b2b|region|shop_name|report_date|country|id  |currency|ordered_products_sale_eur|
+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+-------------------------+
|B07KNDZF8Z|1       |1         |0            |0                |242.25               |73.52                     |0                  |0                      |IT    |Feelino  |2022-02-02 |UK     |6767|GBP     |55.0                     |
|B07KNDZF8Z|1       |1         |0            |0             

In [17]:
joined_df = joined_df.withColumn("ordered_products_sales_b2b_eur", convert_currency(F.col("ordered_products_sales_b2b"), F.col("currency"), F.lit("EUR")))
# Show the result
joined_df.show(5, truncate=False)

+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+-------------------------+------------------------------+
|child_asin|sessions|page_views|units_ordered|units_ordered_b2b|ordered_products_sale|ordered_products_sales_b2b|total_ordered_items|total_ordered_items_b2b|region|shop_name|report_date|country|id  |currency|ordered_products_sale_eur|ordered_products_sales_b2b_eur|
+----------+--------+----------+-------------+-----------------+---------------------+--------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+-------------------------+------------------------------+
|B07KNDZF8Z|1       |1         |0            |0                |242.25               |73.52                     |0                  |0                      |IT    |Feelino  |2022-02-02 |UK     |6767|GBP

In [18]:
joined_df.rdd.getNumPartitions()

1

In [19]:
joined_df.count()

1458

### Calculating the total revenue.

In [20]:
num_partitions = 12 
joined_df = joined_df.repartition(num_partitions)

In [21]:
# The collect() action is used to retrieve all the elements of the DataFrame as an array to the driver program
# [0][0]: This part of the code accesses the first element of the array, which is itself an array due to the collect() function.
# The second [0] is used to access the first (and only) element in the array.
total_revenue = joined_df.agg(sum("ordered_products_sale_eur")).collect()[0][0]
print("Total Revenue for ordered_products_sale_eur:", total_revenue)

Total Revenue for ordered_products_sale_eur: 80190.0


In [22]:
total_revenue = joined_df.agg(sum("ordered_products_sales_b2b_eur")).collect()[0][0]
print("Total Revenue for ordered_products_sales_b2b_eur:", total_revenue)

Total Revenue for ordered_products_sales_b2b_eur: 80190.0


### Calculating total revenue per country.

In [23]:
total_revenue_per_country = joined_df.groupBy("country").sum("ordered_products_sale_eur").withColumnRenamed("sum(ordered_products_sale_eur)", "total_revenue")
total_revenue_per_country.show(10)

+-------+-------------+
|country|total_revenue|
+-------+-------------+
|     UK|       8910.0|
|     ES|       8910.0|
|     PL|       8910.0|
|     FR|       8910.0|
|     SE|       8910.0|
|     DE|       8910.0|
|     IT|       8910.0|
|     NL|       8910.0|
|     TR|       8910.0|
+-------+-------------+



### Calculating total revenue per shop

In [24]:
total_revenue_per_shop = joined_df.groupBy("shop_name").sum("ordered_products_sale_eur").withColumnRenamed("sum(ordered_products_sale_eur)", "total_revenue")
total_revenue_per_shop.show(10)

+---------+-------------+
|shop_name|total_revenue|
+---------+-------------+
|Casa Chic|       6435.0|
|  Feelino|      73755.0|
+---------+-------------+



### Checking how many months of data is present

In [25]:
from pyspark.sql.functions import min, max

# To Find the start and end dates
start_end_dates = joined_df.agg(min("report_date"), max("report_date"))
start_end_dates.show()


+----------------+----------------+
|min(report_date)|max(report_date)|
+----------------+----------------+
|      2022-01-01|      2022-03-31|
+----------------+----------------+



### Calculating total revenue per month

In [26]:
from pyspark.sql.functions import month
final_df_with_month = joined_df.withColumn("month", month("report_date"))
total_revenue_per_month = final_df_with_month.groupBy("month").sum("ordered_products_sale_eur").withColumnRenamed("sum(ordered_products_sale_eur)", "total_revenue")
total_revenue_per_month.show()

+-----+-------------+
|month|total_revenue|
+-----+-------------+
|    2|       9405.0|
|    1|       5445.0|
|    3|      65340.0|
+-----+-------------+



## SQL

### Creating temporary views for the DataFrames

In [27]:
sales_df.createOrReplaceTempView("sales_table")
shop_mapping_df.createOrReplaceTempView("shop_table")

### Converting the sales data to euros using SQL

In [28]:
# Registering the conversion function as a UDF
def conversion_price_function():
    return 55
def conversion_price_function_b2b():
    return 55

spark.udf.register("conversion_price_function", conversion_price_function)
spark.udf.register("conversion_price_function_b2b", conversion_price_function_b2b)

<function __main__.conversion_price_function_b2b()>

In [29]:
# Registering the conversion function as a UDF
def conversion_price_function_euro(from_currency, to_currency):
    if from_currency == "euro" and to_currency == "euro":
        return 1.0  # If the conversion is from euro to euro, the price remains the same
    elif from_currency == "usd" and to_currency == "euro":
        return 0.74  # Assuming 1 USD = 0.74 EUR
    elif from_currency == "gbp" and to_currency == "euro":
        return 0.91  # Assuming 1 GBP = 0.91 EUR
    # Add more currency conversion logic here
    else:
        return 55 

spark.udf.register("conversion_price_function_euro", conversion_price_function_euro)

# SQL query with the registered UDF
sql_euro_conversion_df = spark.sql("""
SELECT
    st.child_asin,
    st.sessions,
    st.page_views,
    st.units_ordered,
    st.units_ordered_b2b,
    st.ordered_products_sale,
    st.ordered_products_sale * conversion_price_function_euro(am.currency, 'euro') AS ordered_products_sale_eur,
    st.ordered_products_sales_b2b,
    st.ordered_products_sales_b2b * conversion_price_function_euro(am.currency, 'euro') AS ordered_products_sales_b2b_eur,
    st.total_ordered_items,
    st.total_ordered_items_b2b,
    st.region,
    st.shop_name,
    st.report_date,
    am.country,
    am.id,
    am.currency
FROM
    sales_table st
LEFT JOIN
    shop_table am
ON
    st.shop_name = am.shop_name
    WHERE am.country IS NOT NULL AND am.id IS NOT NULL AND am.currency IS NOT NULL
""")

sql_euro_conversion_df.show(5)


+----------+--------+----------+-------------+-----------------+---------------------+-------------------------+--------------------------+------------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+
|child_asin|sessions|page_views|units_ordered|units_ordered_b2b|ordered_products_sale|ordered_products_sale_eur|ordered_products_sales_b2b|ordered_products_sales_b2b_eur|total_ordered_items|total_ordered_items_b2b|region|shop_name|report_date|country|  id|currency|
+----------+--------+----------+-------------+-----------------+---------------------+-------------------------+--------------------------+------------------------------+-------------------+-----------------------+------+---------+-----------+-------+----+--------+
|B07KNDZF8Z|       1|         1|            0|                0|               242.25|                 13323.75|                     73.52|                        4043.6|                  0|            

### Calculate the total revenue using SQL

In [30]:
sql_total_revenue_df = spark.sql("""
SELECT
    SUM(ordered_products_sale * conversion_price_function()) AS total_revenue
FROM
    (SELECT
        ordered_products_sale
    FROM
        sales_table st
    LEFT JOIN
        shop_table am
    ON
        st.shop_name = am.shop_name) subquery
""")
sql_total_revenue_df.show()

+-------------------+
|      total_revenue|
+-------------------+
|5.202110815000001E7|
+-------------------+



### Calculating the total revenue per country using SQL

In [31]:
sql_total_revenue_per_country_df = spark.sql("""
SELECT
    country,
    SUM(ordered_products_sale * conversion_price_function()) AS total_revenue
FROM
    (SELECT
        st.ordered_products_sale,
        am.country
    FROM
        sales_table st
    LEFT JOIN
        shop_table am
    ON
        st.shop_name = am.shop_name
        WHERE am.country IS NOT NULL) subquery
GROUP BY
    country
""")
sql_total_revenue_per_country_df.show()

+-------+------------------+
|country|     total_revenue|
+-------+------------------+
|     UK|1348578.0000000002|
|     ES|1348578.0000000002|
|     FR|1348578.0000000002|
|     PL|1348578.0000000002|
|     DE|1348578.0000000002|
|     NL|1348578.0000000002|
|     TR|1348578.0000000002|
|     SE|1348578.0000000002|
|     IT|1348578.0000000002|
+-------+------------------+



### Calculate the total revenue per shop using SQL

In [32]:
sql_total_revenue_per_shop_df = spark.sql("""
SELECT
    shop_name,
    SUM(ordered_products_sale * conversion_price_function()) AS total_revenue
FROM
    (SELECT
        st.ordered_products_sale,
        st.shop_name
    FROM
        sales_table st
    LEFT JOIN
        shop_table am
    ON
        st.shop_name = am.shop_name) subquery
GROUP BY
    shop_name
""")
sql_total_revenue_per_shop_df.show()

+--------------------+--------------------+
|           shop_name|       total_revenue|
+--------------------+--------------------+
|     Elektronik-Star|   2225633.850000001|
|Klarstein Deutsch...|          1713171.35|
|    skullcap-helmets|  1048009.6000000001|
|     BBG Superlunary|            15089.25|
|BBG The Friendly ...| 2.501761405000001E7|
|BBG Scandinavian Hub|            17906.35|
| BBG Zelite-Infinity|  22429.550000000003|
| Berlin Brands Group|             3284.05|
|         BBG BeerCup|  1950539.8000000005|
|BBG Gramercy Kitc...|  2211898.6999999997|
|             Feelino|1.1055453750000034E7|
|            SLIMPURO|   5642200.850000002|
|    BBG Spielehelden|            16128.75|
|           Casa Chic|  1081748.2499999974|
+--------------------+--------------------+

